# Load Models and Embeddings

In [1]:
!pip install langchain_groq langchain_huggingface faiss-cpu faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [3]:
# Load environment variables and models
from google.colab import userdata
groq_token = userdata.get('GROQ_API_KEY')

"""
Comment the above line and uncomment the below line to use your own GROQ API key.
"""
#groq_token = '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'

# Loading the LLM model (Llama 3.1)
from langchain_groq import ChatGroq
groq_model = ChatGroq(api_key=groq_token, model="llama-3.1-8b-instant")

# Loading the HuggingFace embeddings model
from langchain_huggingface import HuggingFaceEmbeddings
embeds = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Define Knowledge Base and Save to File

In [4]:
import os

# Function to save the knowledge base to a file
def save_knowledge_base(data, file_path):
    # Extract directory from the file path
    directory = os.path.dirname(file_path)

    # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists.")

    # Write the knowledge base data to the file
    with open(file_path, "w") as file:
        for entry in data:
            file.write(entry + "\n")
        print(f"Knowledge base saved to {file_path}")

# Define the knowledge base content
knowledge_base = [
    "Category 1 - Login Issues - Login issues often occur due to incorrect passwords or account lockouts.",
    "Category 2 - App Functionality - App crashes can be caused by outdated software or device incompatibility.",
    "Category 3 - Billing - Billing discrepancies may result from processing errors or duplicate transactions.",
    "Category 4 - Account Management - Account management includes tasks such as changing profile information, linking social media accounts, and managing privacy settings.",
    "Category 5 - Performance Issues - Performance issues can be related to device specifications, network connectivity, or app optimization."
]

# Path to store the knowledge base
kb_file_path = "Data/kb_file.txt"

# Save the knowledge base to the file
save_knowledge_base(knowledge_base, kb_file_path)


Directory 'Data' created.
Knowledge base saved to Data/kb_file.txt


# Load and Split the Knowledge Base into Chunks

In [5]:
!pip install langchain_community langchain_text_splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.1 MB/s eta 0:00:00


In [27]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Function to load and split documents
def load_and_split_docs(file_path, chunk_size=200):
    loader = TextLoader(file_path)
    docs = loader.load()  # Loading documents from the file
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size)
    return splitter.split_documents(docs)

# Load and split the knowledge base into chunks
kb_chunks = load_and_split_docs(kb_file_path)

In [28]:
kb_chunks

[Document(metadata={'source': 'Data/kb_file.txt'}, page_content='Category 1 - Login Issues - Login issues often occur due to incorrect passwords or account lockouts.'),
 Document(metadata={'source': 'Data/kb_file.txt'}, page_content='Category 2 - App Functionality - App crashes can be caused by outdated software or device incompatibility.'),
 Document(metadata={'source': 'Data/kb_file.txt'}, page_content='Category 3 - Billing - Billing discrepancies may result from processing errors or duplicate transactions.'),
 Document(metadata={'source': 'Data/kb_file.txt'}, page_content='Category 4 - Account Management - Account management includes tasks such as changing profile information, linking social media accounts, and managing privacy settings.'),
 Document(metadata={'source': 'Data/kb_file.txt'}, page_content='Category 5 - Performance Issues - Performance issues can be related to device specifications, network connectivity, or app optimization.')]

# Create a Vector Store

In [29]:
from langchain_community.vectorstores import FAISS

# Function to create a vector store
def create_vector_store(documents, embeddings_model):
    return FAISS.from_documents(documents=documents, embedding=embeddings_model)

# Create the vector store using the knowledge base chunks and embeddings model
kb_vectorstore = create_vector_store(kb_chunks, embeds)

# Define the System Prompt for Question-Answering

In [30]:
# Importing the ChatPromptTemplate for prompt creation
from langchain_core.prompts import ChatPromptTemplate

# System prompt providing instructions and context
guidelines_prompt = (
    """
### Guidelines

Background Information:
{context}

Review the provided background information, which includes category labels and their descriptions. Then, follow these steps:

1. Analyze the given input text.
2. Categorize the input into one of these categories:
   - Category 1 - Login Issues
   - Category 2 - App Functionality
   - Category 3 - Billing
   - Category 4 - Account Management
   - Category 5 - Performance Issues

3. If the input is unrelated to the background information or you're unsure of the classification, respond with 'I don't know'.

4. Provide only the category label as your response, without any additional text.
    """
)

qa_template = ChatPromptTemplate.from_messages(
    [
        ("system", guidelines_prompt),
        ("human", "{input}"),
    ]
)


# Build the Retrieval-Augmented Generation Chain

In [35]:
# Importing required function to create QA chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Create the QA chain using the vector store and prompt template
qa_chain = create_stuff_documents_chain(groq_model, qa_template)

# Create the retriever for the vector store
retriever = kb_vectorstore.as_retriever(k=3)

# Create the retrieval chain combining vector store and QA chain
rag_chain = create_retrieval_chain(retriever, qa_chain)

# Classify Support Tickets

In [37]:
# Function to classify support tickets using the RAG model
def classify_support_tickets(tickets, rag_chain):
    classified_tickets = []

    # Process each ticket and store the classification result
    for ticket in tickets:
        # Extract the ticket text
        ticket_text = ticket['text']

        # Invoke the RAG chain to classify the ticket
        response = rag_chain.invoke({"input": ticket_text})

        # Append the classification result
        classified_tickets.append({"ticket": ticket_text,"classification": response['answer']})

    return classified_tickets

# Function to display classified tickets in a formatted way
def display_classified_tickets(classified_tickets):
    print("\n--- Classified Support Tickets ---\n")
    for i, ticket_info in enumerate(classified_tickets, 1):
        print(f"Ticket {i}:")
        print(f"   Text: {ticket_info['ticket']}")
        print(f"   Classification: {ticket_info['classification']}")
        print()

# Example tickets
support_tickets = [
    {"text": "My account login is not working. I've tried resetting my password twice."},
    {"text": "The app crashes every time I try to upload a photo."},
    {"text": "I was charged twice for my last subscription payment."},
    {"text": "I can't find the option to change my profile picture."},
    {"text": "The video playback is very laggy on my device."}
]

# Classify the support tickets using the RAG chain
classified_tickets = classify_support_tickets(support_tickets, rag_chain)

# Display the classified tickets
display_classified_tickets(classified_tickets)


--- Classified Support Tickets ---

Ticket 1:
   Text: My account login is not working. I've tried resetting my password twice.
   Classification: Category 1 - Login Issues

Ticket 2:
   Text: The app crashes every time I try to upload a photo.
   Classification: Category 2 - App Functionality

Ticket 3:
   Text: I was charged twice for my last subscription payment.
   Classification: Category 3 - Billing

Ticket 4:
   Text: I can't find the option to change my profile picture.
   Classification: Category 4 - Account Management

Ticket 5:
   Text: The video playback is very laggy on my device.
   Classification: Category 5 - Performance Issues

